title: Smart customers   
author: Fabio Schmidt-Fischbach, Maria Hock 
date: 2021-03-10   
region: EU   
link: https://docs.google.com/presentation/d/1r0i35Q1CkI5czr3K8csW3mq73o-EtZGqdo-lDxhVsTU/edit?usp=sharing   
tags: memberships, amazon, retention, growth, partnership, metal, italy, spain    
summary: This is a collaborative piece written by Maria Hock (UX) and Fabio Schmidt-Fischabch (data) trying to understand why people choose smart and how they use our app afterwards.    
research_type: Data Deep Dives, User Research

## Analysing early tx behaviour using clusters.

In [2]:
import pandas as pd
import altair as alt
import numpy as np

In [7]:
path = "pca_35days_ALL_OLDcohort_RESULTS_FEB202021_N329K_Relabeled_clusters.csv"

In [6]:
df.columns

Index(['Unnamed: 0', 'user_created', 'user_id', 'market', 'is_expat',
       'age_group', 'last_click_source', 'has_overdraft_enabled', 'is_dunning',
       'product_id', 'is_premium', 'su_product_id', 'su_is_premium',
       'kycc_cohort', 'kycc_card_days', 'kyc_ct_days', 'n_pt', 'n_dt', 'n_ft',
       'n_dd', 'n_tub', 'n_ct', 'n_cash26', 'n_stripe', 'n_atm', 'n_ecomm',
       'n_cardpresent', 'n_mobile', 'n_spaces_ct', 'n_spaces_dt', 'n_wu',
       'n_act_total', 'n_int_total', 'n_total', 'n_inter', 'n_dom', 'n_intra',
       'n_inter_atm', 'n_dom_atm', 'n_intra_atm', 'pt_amt', 'dt_amt', 'ft_amt',
       'dd_amt', 'tub_amt', 'ct_amt', 'cash26_amt', 'stripe_amt', 'atm_amt',
       'ecomm_amt', 'cardpresent_amt', 'mobile_amt', 'spaces_ct_amt',
       'spaces_dt_amt', 'wu_amt', 'act_total_amt', 'int_total_amt',
       'total_amt', 'inter_sum', 'dom_sum', 'intra_sum', 'inter_atm_sum',
       'dom_atm_sum', 'intra_atm_sum', 'primary_bal_avg', 'space_bal_avg',
       'day35_bal', 'act_txns

## Create new cluster labels 

In [50]:
df = pd.read_csv(path)

di = {
    "E1": "Early Spaces Power Users",
    "E2": "Early International Travelers",
    "E3": "Early Euro Travelers",
    "E5": "Early Domestic Spenders",
    "E4": "Early Cash26ers",
    "E6": "Early Mobile Spenders",
    "E7": "Early Online Spenders",
    "E8": "Early Barely Active",
    "E9": "Early Cash Spenders",
    "E10": "Eary Holding Account",
    "E11": "Early friend referral & run",
    "E12": "Early primary account user",
}

rank = {
    "Early primary account user": 0,
    "Early Spaces Power users": 1,
    "Early International Travelers": 2,
    "Early Euro Travelers": 3,
    "Early Cash26ers": 4,
    "Early Domestic Spenders": 5,
    "Early Mobile spenders": 6,
    "Early Online Spenders": 7,
    "Early Barely Active": 8,
    "Early Cash Spenders": 9,
    "Early Holding account": 10,
    "Early friend referral & run": 11,
}

high = {
    "Early primary account user": 1,
    "Early Spaces Power users": 1,
    "Early International Travelers": 1,
    "Early Euro Travelers": 1,
    "Early Cash26ers": 1,
    "Early Domestic Spenders": 1,
    "Early Mobile spenders": 1,
    "Early Online Spenders": 0,
    "Early Barely Active": 0,
    "Early Cash Spenders": 0,
    "Early Holding account": 0,
    "Early friend referral & run": 0,
}


df["cluster_new"] = df["cluster_new"].map(di)

df.cluster_new.unique()

array(['Early Mobile Spenders', 'Early Spaces Power Users',
       'Eary Holding Account', 'Early Domestic Spenders',
       'Early primary account user', 'Early International Travelers',
       'Early Barely Active', 'Early Euro Travelers',
       'Early Cash Spenders', 'Early Online Spenders', 'Early Cash26ers',
       'Early friend referral & run'], dtype=object)

In [57]:
df = pd.read_csv(path)

di = {
    "E1": "Early Spaces Power Users",
    "E2": "Early International Travelers",
    "E3": "Early Euro Travelers",
    "E5": "Early Domestic Spenders",
    "E4": "Early Cash26ers",
    "E6": "Early Mobile Spenders",
    "E7": "Early Online Spenders",
    "E8": "Early Barely Active",
    "E9": "Early Cash Spenders",
    "E10": "Early Holding Account",
    "E11": "Early friend referral & run",
    "E12": "Early primary account user",
}

rank = {
    "Early primary account user": 0,
    "Early Spaces Power Users": 1,
    "Early International Travelers": 2,
    "Early Euro Travelers": 3,
    "Early Cash26ers": 4,
    "Early Domestic Spenders": 5,
    "Early Mobile Spenders": 6,
    "Early Online Spenders": 7,
    "Early Barely Active": 8,
    "Early Cash Spenders": 9,
    "Early Holding Account": 10,
    "Early friend referral & run": 11,
}

high = {
    "Early primary account user": 1,
    "Early Spaces Power Users": 1,
    "Early International Travelers": 1,
    "Early Euro Travelers": 1,
    "Early Cash26ers": 1,
    "Early Domestic Spenders": 1,
    "Early Mobile Spenders": 1,
    "Early Online Spenders": 0,
    "Early Barely Active": 0,
    "Early Cash Spenders": 0,
    "Early Holding Account": 0,
    "Early friend referral & run": 0,
}


df["cluster_new"] = df["cluster_new"].map(di)
df = df.loc[df["kycc_cohort"] == "2020-12-01", :]
df["rank"] = df["cluster_new"].map(rank)
df["high"] = df["cluster_new"].map(high)

df = df.groupby(["cluster_new", "rank", "high"])["user_id"].agg("nunique").reset_index()
df["perc"] = 100 * df["user_id"] / df["user_id"].sum()

# Configure common options
alt.Chart(df).mark_bar().encode(
    x=alt.X(
        "cluster_new:N",
        sort=alt.SortField("rank", order="ascending"),
        axis=alt.Axis(title="Cluster"),
    ),
    y=alt.Y("perc:Q", axis=alt.Axis(title="% of Nov/Dec 2020 cohort")),
    color=alt.Color("high:N", legend=alt.Legend(title="High activity")),
).properties(width=500, height=500, title="% of cohort part of cluster").configure_axis(
    labelFontSize=13, titleFontSize=15
)

alt.Chart(...)

In [58]:
df = pd.read_csv(path)

di = {
    "E1": "Early Spaces Power Users",
    "E2": "Early International Travelers",
    "E3": "Early Euro Travelers",
    "E5": "Early Domestic Spenders",
    "E4": "Early Cash26ers",
    "E6": "Early Mobile Spenders",
    "E7": "Early Online Spenders",
    "E8": "Early Barely Active",
    "E9": "Early Cash Spenders",
    "E10": "Early Holding Account",
    "E11": "Early friend referral & run",
    "E12": "Early primary account user",
}

rank = {
    "Early primary account user": 0,
    "Early Spaces Power Users": 1,
    "Early International Travelers": 2,
    "Early Euro Travelers": 3,
    "Early Cash26ers": 4,
    "Early Domestic Spenders": 5,
    "Early Mobile Spenders": 6,
    "Early Online Spenders": 7,
    "Early Barely Active": 8,
    "Early Cash Spenders": 9,
    "Early Holding Account": 10,
    "Early friend referral & run": 11,
}

high = {
    "Early primary account user": 1,
    "Early Spaces Power Users": 1,
    "Early International Travelers": 1,
    "Early Euro Travelers": 1,
    "Early Cash26ers": 1,
    "Early Domestic Spenders": 1,
    "Early Mobile Spenders": 1,
    "Early Online Spenders": 0,
    "Early Barely Active": 0,
    "Early Cash Spenders": 0,
    "Early Holding Account": 0,
    "Early friend referral & run": 0,
}


df["cluster_new"] = df["cluster_new"].map(di).fillna(df["cluster_new"])
df = df.loc[df["kycc_cohort"] == "2020-12-01", :]
df["rank"] = df["cluster_new"].map(rank)
df["high"] = df["cluster_new"].map(high)


df = (
    df.groupby(["cluster_new", "rank", "high"])["n_act_total"].agg("mean").reset_index()
)

# Configure common options
alt.Chart(df).mark_bar().encode(
    x=alt.X(
        "cluster_new:N",
        sort=alt.SortField("rank", order="ascending"),
        axis=alt.Axis(title="Cluster"),
    ),
    y=alt.Y("n_act_total:Q", axis=alt.Axis(title="Avg. # of txs in first 35 days")),
    color=alt.Color("high:N", legend=alt.Legend(title="High activity")),
).properties(width=400, height=400, title="# of txs (35d)").configure_axis(
    labelFontSize=13, titleFontSize=15
)

alt.Chart(...)

In [61]:
df = pd.read_csv(path)

di = {
    "E1": "Early Spaces Power Users",
    "E2": "Early International Travelers",
    "E3": "Early Euro Travelers",
    "E5": "Early Domestic Spenders",
    "E4": "Early Cash26ers",
    "E6": "Early Mobile Spenders",
    "E7": "Early Online Spenders",
    "E8": "Early Barely Active",
    "E9": "Early Cash Spenders",
    "E10": "Early Holding Account",
    "E11": "Early friend referral & run",
    "E12": "Early primary account user",
}

rank = {
    "Early primary account user": 0,
    "Early Spaces Power Users": 1,
    "Early International Travelers": 2,
    "Early Euro Travelers": 3,
    "Early Cash26ers": 4,
    "Early Domestic Spenders": 5,
    "Early Mobile Spenders": 6,
    "Early Online Spenders": 7,
    "Early Barely Active": 8,
    "Early Cash Spenders": 9,
    "Early Holding Account": 10,
    "Early friend referral & run": 11,
}

high = {
    "Early primary account user": 1,
    "Early Spaces Power Users": 1,
    "Early International Travelers": 1,
    "Early Euro Travelers": 1,
    "Early Cash26ers": 1,
    "Early Domestic Spenders": 1,
    "Early Mobile Spenders": 1,
    "Early Online Spenders": 0,
    "Early Barely Active": 0,
    "Early Cash Spenders": 0,
    "Early Holding Account": 0,
    "Early friend referral & run": 0,
}


df["cluster_new"] = df["cluster_new"].map(di).fillna(df["cluster_new"])
df = df.loc[df["kycc_cohort"] == "2020-12-01", :]
df["rank"] = df["cluster_new"].map(rank)
df["high"] = df["cluster_new"].map(high)


# only certain features
df = df.loc[
    :,
    [
        "cluster_new",
        "user_id",
        "rank",
        "high",
        "n_atm",
        "cardpresent_amt",
        "primary_bal_avg",
        "space_bal_avg",
        "n_pt",
        "n_dd",
        "n_dt",
        "n_ct",
        "n_inter",
        "n_intra",
        "login_days",
        "n_wu",
        "tub_amt",
        "n_cash26",
        "n_ecomm",
        "n_mobile",
    ],
]
df = df.melt(id_vars=["user_id", "rank", "high", "cluster_new"])

df = (
    df.groupby(["cluster_new", "rank", "high", "variable"])["value"]
    .agg("mean")
    .reset_index()
)

df["norm"] = df["value"] / df.groupby(["variable"])["value"].transform("max")

# only high activity
# df = df.loc[df["high"]==1,:]

# Configure common options
base = alt.Chart(df).encode(
    alt.Y("variable:O", scale=alt.Scale(paddingInner=0)),
    alt.X(
        "cluster_new:O",
        scale=alt.Scale(paddingInner=0),
        sort=alt.SortField("rank", order="ascending"),
        axis=alt.Axis(title="Cluster"),
    ),
)

# Configure heatmap
heatmap = (
    base.mark_rect()
    .encode(
        color=alt.Color(
            "norm:Q",
            scale=alt.Scale(scheme="viridis"),
            legend=alt.Legend(direction="horizontal"),
        )
    )
    .properties(height=600, width=600)
    .configure_axis(labelFontSize=15, titleFontSize=15)
)


# Draw the chart
heatmap

alt.Chart(...)

In [63]:
df = pd.read_csv(path)

di = {
    "E1": "Early Spaces Power Users",
    "E2": "Early International Travelers",
    "E3": "Early Euro Travelers",
    "E5": "Early Domestic Spenders",
    "E4": "Early Cash26ers",
    "E6": "Early Mobile Spenders",
    "E7": "Early Online Spenders",
    "E8": "Early Barely Active",
    "E9": "Early Cash Spenders",
    "E10": "Early Holding Account",
    "E11": "Early friend referral & run",
    "E12": "Early primary account user",
}

rank = {
    "Early primary account user": 0,
    "Early Spaces Power Users": 1,
    "Early International Travelers": 2,
    "Early Euro Travelers": 3,
    "Early Cash26ers": 4,
    "Early Domestic Spenders": 5,
    "Early Mobile Spenders": 6,
    "Early Online Spenders": 7,
    "Early Barely Active": 8,
    "Early Cash Spenders": 9,
    "Early Holding Account": 10,
    "Early friend referral & run": 11,
}

high = {
    "Early primary account user": 1,
    "Early Spaces Power Users": 1,
    "Early International Travelers": 1,
    "Early Euro Travelers": 1,
    "Early Cash26ers": 1,
    "Early Domestic Spenders": 1,
    "Early Mobile Spenders": 1,
    "Early Online Spenders": 0,
    "Early Barely Active": 0,
    "Early Cash Spenders": 0,
    "Early Holding Account": 0,
    "Early friend referral & run": 0,
}


df["cluster_new"] = df["cluster_new"].map(di).fillna(df["cluster_new"])
df = df.loc[df["kycc_cohort"] == "2020-12-01", :]
df["rank"] = df["cluster_new"].map(rank)
df["high"] = df["cluster_new"].map(high)


df["n_spaces"] = df["n_spaces_ct"] + df["n_spaces_dt"]

df = df[
    [
        "n_spaces",
        "space_bal_avg",
        "primary_bal_avg",
        "ct_amt",
        "inter_sum",
        "intra_sum",
        "dom_sum",
        "cardpresent_amt",
        "n_pt",
        "dt_amt",
        "cluster_new",
        "user_id",
        "rank",
    ]
]


df = df.melt(id_vars=["user_id", "rank", "cluster_new"])

df = df.groupby(["cluster_new", "rank", "variable"])["value"].agg("mean").reset_index()


df = df.loc[
    df["variable"].isin(
        [
            "n_spaces",
            "space_bal_avg",
            "primary_bal_avg",
            "ct_amt",
            "inter_sum",
            "intra_sum",
            "dom_sum",
            "cardpresent_amt",
            "n_pt",
            "dt_amt",
        ]
    ),
    :,
]

alt.Chart(df).mark_bar().encode(
    x=alt.X(
        "cluster_new:N",
        sort=alt.SortField("rank", order="ascending"),
        axis=alt.Axis(title="Cluster"),
    ),
    y="value:Q",
    color=alt.Color("cluster_new:N", scale=alt.Scale(scheme="category20")),
).properties(width=300, height=100).facet(facet="variable:N", columns=2).resolve_scale(
    y="independent"
).configure_axis(
    labelFontSize=15, titleFontSize=15
)

alt.FacetChart(...)

In [ ]:
query = """
select user_id, dbt.zrh_user_product.product_id, status
from dbt.zrh_user_product 
inner join dbt.zrh_users using(user_created) 
where enter_reason = 'SIGNUP'
group by 1,2,3
"""

In [ ]:
results = "cluster_35days_10K_4UserClusters_Smart_ClusteringResults.csv"
df = pd.read_csv(results)
prod = pd.read_csv("product.csv")
df = df.merge(prod, on="user_id", how="left")
df["product_id"] = df.loc[df["product_id"].isna() == True, "product_id"] = "STANDARD"

## User behaviour over time. 

In [64]:
df = pd.read_csv(path)

di = {
    "E1": "Early Spaces Power Users",
    "E2": "Early International Travelers",
    "E3": "Early Euro Travelers",
    "E5": "Early Domestic Spenders",
    "E4": "Early Cash26ers",
    "E6": "Early Mobile Spenders",
    "E7": "Early Online Spenders",
    "E8": "Early Barely Active",
    "E9": "Early Cash Spenders",
    "E10": "Early Holding Account",
    "E11": "Early friend referral & run",
    "E12": "Early primary account user",
}

rank = {
    "Early primary account user": 0,
    "Early Spaces Power Users": 1,
    "Early International Travelers": 2,
    "Early Euro Travelers": 3,
    "Early Cash26ers": 4,
    "Early Domestic Spenders": 5,
    "Early Mobile Spenders": 6,
    "Early Online Spenders": 7,
    "Early Barely Active": 8,
    "Early Cash Spenders": 9,
    "Early Holding Account": 10,
    "Early friend referral & run": 11,
}

high = {
    "Early primary account user": 1,
    "Early Spaces Power Users": 1,
    "Early International Travelers": 1,
    "Early Euro Travelers": 1,
    "Early Cash26ers": 1,
    "Early Domestic Spenders": 1,
    "Early Mobile Spenders": 1,
    "Early Online Spenders": 0,
    "Early Barely Active": 0,
    "Early Cash Spenders": 0,
    "Early Holding Account": 0,
    "Early friend referral & run": 0,
}


df["cluster_new"] = df["cluster_new"].map(di).fillna(df["cluster_new"])
df["rank"] = df["cluster_new"].map(rank)
df["high"] = df["cluster_new"].map(high)

df = (
    df.groupby(["cluster_new", "high", "rank", "kycc_cohort"])["user_id"]
    .agg("nunique")
    .reset_index()
)

df["perc"] = round(
    100 * df["user_id"] / df.groupby(["kycc_cohort"])["user_id"].transform("sum")
)

df = df.sort_values(by=["rank"], ascending=True)

# Configure common options
alt.Chart(df).mark_line().encode(
    x=alt.X("kycc_cohort:T", axis=alt.Axis(title="KYC cohort")),
    y=alt.Y("perc:Q", axis=alt.Axis(title="% of cohort")),
    color=alt.Color("cluster_new:N", legend=alt.Legend(title="Cohort")),
    column="high:N",
).properties(width=400, height=400, title="# of txs (35d)").configure_axis(
    labelFontSize=13, titleFontSize=15
)

alt.Chart(...)

## understand product choices

In [66]:
df = pd.read_csv(path)

di = {
    "E1": "Early Spaces Power Users",
    "E2": "Early International Travelers",
    "E3": "Early Euro Travelers",
    "E5": "Early Domestic Spenders",
    "E4": "Early Cash26ers",
    "E6": "Early Mobile Spenders",
    "E7": "Early Online Spenders",
    "E8": "Early Barely Active",
    "E9": "Early Cash Spenders",
    "E10": "Early Holding Account",
    "E11": "Early friend referral & run",
    "E12": "Early primary account user",
}

rank = {
    "Early primary account user": 0,
    "Early Spaces Power Users": 1,
    "Early International Travelers": 2,
    "Early Euro Travelers": 3,
    "Early Cash26ers": 4,
    "Early Domestic Spenders": 5,
    "Early Mobile Spenders": 6,
    "Early Online Spenders": 7,
    "Early Barely Active": 8,
    "Early Cash Spenders": 9,
    "Early Holding Account": 10,
    "Early friend referral & run": 11,
}

high = {
    "Early primary account user": 1,
    "Early Spaces Power Users": 1,
    "Early International Travelers": 1,
    "Early Euro Travelers": 1,
    "Early Cash26ers": 1,
    "Early Domestic Spenders": 1,
    "Early Mobile Spenders": 1,
    "Early Online Spenders": 0,
    "Early Barely Active": 0,
    "Early Cash Spenders": 0,
    "Early Holding Account": 0,
    "Early friend referral & run": 0,
}


df["cluster_new"] = df["cluster_new"].map(di).fillna(df["cluster_new"])
df["rank"] = df["cluster_new"].map(rank)
df["high"] = df["cluster_new"].map(high)

df = df.loc[df["kycc_cohort"] == "2020-12-01", :]

df = (
    df.groupby(["cluster_new", "rank", "product_id"])["user_id"]
    .agg("nunique")
    .reset_index()
)

df["perc"] = round(
    100 * df["user_id"] / df.groupby(["product_id"])["user_id"].transform("sum"), 0
)


# Configure common options
base = alt.Chart(
    df.loc[
        df["product_id"].isin(
            ["STANDARD", "BLACK_CARD_MONTHLY", "METAL_CARD_MONTHLY", "SMART"]
        ),
        :,
    ]
).encode(
    alt.Y(
        "product_id:O",
        scale=alt.Scale(paddingInner=0),
        axis=alt.Axis(title="Product at signup"),
    ),
    alt.X(
        "cluster_new:O",
        scale=alt.Scale(paddingInner=0),
        axis=alt.Axis(title="Cluster"),
        sort=alt.SortField("rank", order="ascending"),
    ),
)

# Configure heatmap
heatmap = (
    base.mark_rect()
    .encode(
        color=alt.Color(
            "perc:Q",
            scale=alt.Scale(scheme="viridis"),
            legend=alt.Legend(direction="horizontal"),
        )
    )
    .properties(height=300, width=600)
)
# Configure text
text = base.mark_text(baseline="middle").encode(
    text="perc:Q",
    color=alt.condition(
        alt.datum.num_cars > 100, alt.value("black"), alt.value("white")
    ),
)

# Draw the chart
heatmap + text

alt.LayerChart(...)

## Money management as a pull

In [4]:
query = """


select sf.user_id, channel as feature, NULL::text as detail, count(distinct sf.id) as variable 
from dbt.sf_all_contacts as sf 
inner join dbt.zrh_users as cmd on sf.user_id = cmd.user_id and user_created >= '2020-11-01'
where channel in ('call','email') and datediff('days',cmd.kyc_first_completed::date, sf.contact_date::date)<= 35
group by 1,2,3 

union all 

select user_id,  se_action as feature, partnership_name as detail, count(1) as variable 
from dbt.gp_explore_tab_events as gp 
inner join dbt.zrh_users using (user_created) 
where user_created >= '2020-11-01' and datediff('days',dbt.zrh_users.kyc_first_completed::date, gp.collector_tstamp::date)<= 35
group by 1,2,3 

union all 

select user_id, 'statistics' as feature, NULL::text as detail, count(distinct collector_tstamp::date) as variable
from dbt.zrh_snowplow_statistics 
inner join dbt.zrh_users using (user_created)
where user_created >= '2020-11-01' and datediff('days',dbt.zrh_users.kyc_first_completed::date, collector_tstamp::date)<= 35
group by 1,2,3

union all 

select user_id, 'round-ups' as feature, NULL::text as detail, count(case when status = 'ENABLED' then 1 else null end) as variable 
from cn_automated_rule
where user_created >= '2020-11-01' 
group by 1,2,3 

union all 

select user_id, feature, null::text as detail, count(1) 
from dbt.zrh_spaces_rows 
inner join dbt.zrh_users using (user_created)
where month = '2021-01-01' and feature in ('n_reg_spaces', 'n_shared_spaces' ) and user_created >= '2020-11-01'
group by 1,2,3 

union all 

select user_id, 'atm' as feature, null::text as detail, sum(n_card_atm)
from dbt.zrh_txn_day  
inner join dbt.zrh_users using (user_created)
where user_created >= '2020-11-01'  and datediff('days',dbt.zrh_users.kyc_first_completed::date, txn_date::date) <= 35
group by 1,2,3 

union all 

select user_id, case when se_action = 'premium.membership_overview_screen.viewed' then 'Benefits screen' else 'Insurance coverage' end as feature, NULL::text as detail, count(distinct collector_tstamp::date) as variable
from dbt.zrh_solutions_fe  
inner join dbt.zrh_users using (user_created)
where user_created >= '2020-11-01'
	and se_action in ('premium.membership_overview_screen.viewed', 'premium.membership_insurance_coverage_screen.viewed')
	and datediff('days',dbt.zrh_users.kyc_first_completed::date, collector_tstamp::date) <= 35
group by 1,2,3

"""

In [71]:
prem = pd.read_csv("prem_features.csv")

prem = (
    pd.pivot_table(prem, index=["user_id"], values=["variable"], columns=["feature"])
    .reset_index()
    .fillna(0)
)

prem = pd.melt(
    prem,
    id_vars=[
        "user_id",
    ],
)


df = pd.read_csv(path)

di = {
    "E1": "Early Spaces Power Users",
    "E2": "Early International Travelers",
    "E3": "Early Euro Travelers",
    "E5": "Early Domestic Spenders",
    "E4": "Early Cash26ers",
    "E6": "Early Mobile Spenders",
    "E7": "Early Online Spenders",
    "E8": "Early Barely Active",
    "E9": "Early Cash Spenders",
    "E10": "Early Holding Account",
    "E11": "Early friend referral & run",
    "E12": "Early primary account user",
}

rank = {
    "Early primary account user": 0,
    "Early Spaces Power Users": 1,
    "Early International Travelers": 2,
    "Early Euro Travelers": 3,
    "Early Cash26ers": 4,
    "Early Domestic Spenders": 5,
    "Early Mobile Spenders": 6,
    "Early Online Spenders": 7,
    "Early Barely Active": 8,
    "Early Cash Spenders": 9,
    "Early Holding Account": 10,
    "Early friend referral & run": 11,
}

high = {
    "Early primary account user": 1,
    "Early Spaces Power Users": 1,
    "Early International Travelers": 1,
    "Early Euro Travelers": 1,
    "Early Cash26ers": 1,
    "Early Domestic Spenders": 1,
    "Early Mobile Spenders": 1,
    "Early Online Spenders": 0,
    "Early Barely Active": 0,
    "Early Cash Spenders": 0,
    "Early Holding Account": 0,
    "Early friend referral & run": 0,
}


df["cluster_new"] = df["cluster_new"].map(di).fillna(df["cluster_new"])
df["rank"] = df["cluster_new"].map(rank)
df["high"] = df["cluster_new"].map(high)

df = df.loc[df["kycc_cohort"] == "2020-12-01", :]

prem = prem.merge(df, on="user_id", how="right").fillna(0)

# keep only premium.
prem = prem.loc[
    prem["product_id"].isin(["STANDARD", "BUSINESS_CARD", "FLEX_ACCOUNT_MONTHLY"])
    == False,
    :,
]

# recode to binary
prem.loc[prem["value"] > 0, "value"] = 1

# compute% that used it

prem = (
    prem.groupby(["cluster_new", "rank", "feature"])["value"].agg("mean").reset_index()
)

prem["value"] = round(prem["value"] * 100)

prem = prem.loc[prem["feature"].astype(str) != "0", :]

feature_dic = {
    "Partner benefits": "Visited partner benefits",
    "atm": "Withdrew from ATM",
    "call": "Called CS via phone",
    "n_reg_spaces": "Has active regular space",
    "n_shared_spaces": "Has active shared space",
    "round-ups": "Has active round-up rule",
    "statistics": "Visited statistics",
    "Benefits screen": "Visited benefits screen",
    "Insurance coverage": "Visited insurance coverage",
}
prem["feature"] = prem["feature"].map(feature_dic)

# Configure common options
base = alt.Chart(prem.loc[pd.isnull(prem["feature"]) == False, :]).encode(
    alt.Y("feature:O", scale=alt.Scale(paddingInner=0), axis=alt.Axis(title="Feature")),
    alt.X(
        "cluster_new:O",
        scale=alt.Scale(paddingInner=0),
        axis=alt.Axis(title="Cluster"),
        sort=alt.SortField("rank", order="ascending"),
    ),
)


# Configure heatmap
heatmap = (
    base.mark_rect()
    .encode(
        color=alt.Color(
            "value:Q",
            scale=alt.Scale(scheme="viridis"),
            legend=alt.Legend(direction="horizontal"),
        )
    )
    .properties(height=300, width=700)
)
# Configure text
text = base.mark_text(baseline="middle").encode(text="value:Q")

# Draw the chart
(heatmap + text).configure_axis(labelFontSize=15, titleFontSize=15)

/Users/fabioschmidt-fischbach/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/fabioschmidt-fischbach/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (12,71) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


alt.LayerChart(...)

In [86]:
prem = pd.read_csv("prem_features.csv")

# recode benefits
prem.loc[
    prem["feature"].str.contains("partner") == True, "feature"
] = "Partner benefits"
prem = prem.loc[prem["feature"] != "email", :]


p = pd.read_csv("product.csv")

prem = prem.merge(p, on="user_id", how="left").reset_index()
prem = prem.drop(columns=["detail", "index"])

prem = (
    pd.pivot_table(
        prem,
        index=["product_id", "user_id", "status"],
        values=["variable"],
        columns=["feature"],
    )
    .reset_index()
    .fillna(0)
)

prem = pd.melt(prem, id_vars=["user_id", "product_id", "status"])

# keep only premium.
prem = prem.loc[
    prem["product_id"].isin(["STANDARD", "BUSINESS_CARD", "FLEX_ACCOUNT_MONTHLY"])
    == False,
    :,
]

# recode to binary
prem.loc[prem["value"] > 0, "value"] = 1

# compute% that used it

prem = prem.groupby(["feature"])["value"].agg("mean").reset_index()

prem["value"] = round(prem["value"] * 100)

prem = prem.loc[prem["feature"].astype(str) != "0", :]


feature_dic = {
    "Partner benefits": "Visited partner benefits",
    "atm": "Withdrew from ATM",
    "call": "Called CS via phone",
    "n_reg_spaces": "Has active regular space",
    "n_shared_spaces": "Has active shared space",
    "round-ups": "Has active round-up rule",
    "statistics": "Visited statistics",
    "Benefits screen": "Visited benefits screen",
    "Insurance coverage": "Visited insurance coverage",
}
prem["feature"] = prem["feature"].map(feature_dic)

prem = prem.sort_values(by=["value"])
# Configure common options
alt.Chart(prem.loc[pd.isnull(prem["feature"]) == False, :]).mark_bar().encode(
    alt.Y("feature:O", sort=alt.SortField("value:Q", order="ascending")),
    alt.X(
        "value:Q",
        axis=alt.Axis(title="% of premium users in sample that used feature at all"),
    ),
).configure_axis(labelFontSize=15, titleFontSize=15).properties(
    title="% of premium users that used X within 35d of KYC", width=400, height=400
)

/Users/fabioschmidt-fischbach/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


alt.Chart(...)

In [85]:
prem.head()

,feature,value
1,Visited benefits screen,59.0
2,Visited insurance coverage,20.0
3,Visited partner benefits,25.0
4,Withdrew from ATM,31.0
5,Called CS via phone,11.0


In [85]:
prem = pd.read_csv("prem_features.csv")

prem = (
    pd.pivot_table(prem, index=["user_id"], values=["variable"], columns=["feature"])
    .reset_index()
    .fillna(0)
)

prem = pd.melt(
    prem,
    id_vars=[
        "user_id",
    ],
)


df = pd.read_csv(path)

di = {
    "E1": "Early Spaces Power Users",
    "E2": "Early International Travelers",
    "E3": "Early Euro Travelers",
    "E5": "Early Domestic Spenders",
    "E4": "Early Cash26ers",
    "E6": "Early Mobile Spenders",
    "E7": "Early Online Spenders",
    "E8": "Early Barely Active",
    "E9": "Early Cash Spenders",
    "E10": "Early Holding Account",
    "E11": "Early friend referral & run",
    "E12": "Early primary account user",
}

rank = {
    "Early primary account user": 0,
    "Early Spaces Power Users": 1,
    "Early International Travelers": 2,
    "Early Euro Travelers": 3,
    "Early Cash26ers": 4,
    "Early Domestic Spenders": 5,
    "Early Mobile Spenders": 6,
    "Early Online Spenders": 7,
    "Early Barely Active": 8,
    "Early Cash Spenders": 9,
    "Early Holding Account": 10,
    "Early friend referral & run": 11,
}

high = {
    "Early primary account user": 1,
    "Early Spaces Power Users": 1,
    "Early International Travelers": 1,
    "Early Euro Travelers": 1,
    "Early Cash26ers": 1,
    "Early Domestic Spenders": 1,
    "Early Mobile Spenders": 1,
    "Early Online Spenders": 0,
    "Early Barely Active": 0,
    "Early Cash Spenders": 0,
    "Early Holding Account": 0,
    "Early friend referral & run": 0,
}


df["cluster_new"] = df["cluster_new"].map(di).fillna(df["cluster_new"])
df["rank"] = df["cluster_new"].map(rank)
df["high"] = df["cluster_new"].map(high)

df = df.loc[df["kycc_cohort"] == "2020-12-01", :]

prem = prem.merge(df, on="user_id", how="right").fillna(0)

# keep only premium.
prem = prem.loc[
    prem["product_id"].isin(["STANDARD", "BUSINESS_CARD", "FLEX_ACCOUNT_MONTHLY"])
    == False,
    :,
]

# recode to binary
prem.loc[prem["value"] > 0, "value"] = 1

# compute% that used it

prem = prem.groupby(["product_id", "feature"])["value"].agg("mean").reset_index()

prem["value"] = round(prem["value"] * 100)

prem = prem.loc[prem["feature"].astype(str) != "0", :]

feature_dic = {
    "Partner benefits": "Visited partner benefits",
    "atm": "Withdrew from ATM",
    "call": "Called CS via phone",
    "n_reg_spaces": "Has active regular space",
    "n_shared_spaces": "Has active shared space",
    "round-ups": "Has active round-up rule",
    "statistics": "Visited statistics",
    "Benefits screen": "Visited benefits screen",
    "Insurance coverage": "Visited insurance coverage",
}
prem["feature"] = prem["feature"].map(feature_dic)


product_dic = {
    "BLACK_CARD_MONTHLY": "You",
    "METAL_CARD_MONTHLY": "Metal",
    "SMART": "Smart",
}
prem["product_id"] = prem["product_id"].map(product_dic)


# Configure common options
base = alt.Chart(
    prem.loc[
        prem["product_id"].isin(["You", "Metal", "Smart"])
        & (prem["feature"].isnull() == False),
        :,
    ]
).encode(
    alt.X("feature:O", scale=alt.Scale(paddingInner=0), axis=alt.Axis(title="Feature")),
    alt.Y(
        "product_id:O", scale=alt.Scale(paddingInner=0), axis=alt.Axis(title="Product")
    ),
)


# Configure heatmap
heatmap = (
    base.mark_rect()
    .encode(
        color=alt.Color(
            "value:Q",
            scale=alt.Scale(scheme="viridis"),
            legend=alt.Legend(direction="horizontal"),
        )
    )
    .properties(height=300, width=400)
)
# Configure text
text = base.mark_text(baseline="middle").encode(text="value:Q")

# Draw the chart
(heatmap + text).configure_axis(labelFontSize=15, titleFontSize=15)

/Users/fabioschmidt-fischbach/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/fabioschmidt-fischbach/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (12,71) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


alt.LayerChart(...)